# CSCI 331 — HW 4 & 5 (Subqueries & Table Expressions)

**Database:** WideWorldImporters  
**Student:** Zarrin Cherry | **Group:** 2  

This notebook includes:
- The exact SQL for 10 propositions
- A brief observation about what I saw when executing in DBeaver
- A screenshot of the **actual result** under each query for GitHub preview

_Generated: 2025-10-10 20:56:03_


## Proposition 1 — NOT EXISTS: customers with no orders in 2015

**Observation:** Non-empty: I see ~6 customers — these had zero orders in 2015.

```sql
SELECT C.CustomerID, C.CustomerName
FROM Sales.Customers AS C
WHERE NOT EXISTS
(
    SELECT 1
    FROM Sales.Orders AS O
    WHERE O.CustomerID = C.CustomerID
      AND O.OrderDate >= '20150101' AND O.OrderDate < '20160101'
);
```

**Execution Result (screenshot):**

![](/mnt/data/E9962937-B73D-4A6A-9B20-930107D721F5.jpeg)

## Proposition 2 — Scalar subquery: orders above that customer’s average

**Observation:** Non-empty: many orders exceeded the customer’s own average.

```sql
SELECT O.OrderID, O.CustomerID,
       SUM(OL.UnitPrice * OL.Quantity) AS OrderTotal
FROM Sales.Orders AS O
JOIN Sales.OrderLines AS OL
  ON OL.OrderID = O.OrderID
GROUP BY O.OrderID, O.CustomerID
HAVING SUM(OL.UnitPrice * OL.Quantity) >
(
    SELECT AVG(T.OrderTotal)
    FROM (
        SELECT O2.OrderID, SUM(OL2.UnitPrice * OL2.Quantity) AS OrderTotal
        FROM Sales.Orders AS O2
        JOIN Sales.OrderLines AS OL2 ON OL2.OrderID = O2.OrderID
        WHERE O2.CustomerID = O.CustomerID
        GROUP BY O2.OrderID
    ) AS T
);
```

**Execution Result (screenshot):**

![](/mnt/data/E0B7CE21-ECFA-4A0D-8BFD-7912D70CC462.jpeg)

## Proposition 3 — Double NOT EXISTS (relational division): suppliers who supplied every color

**Observation:** EMPTY: no supplier supplied every color.

```sql
SELECT S.SupplierID, S.SupplierName
FROM Purchasing.Suppliers AS S
WHERE NOT EXISTS
(
    SELECT DISTINCT SI.ColorID
    FROM Warehouse.StockItems AS SI
    WHERE SI.ColorID IS NOT NULL
    AND NOT EXISTS
    (
        SELECT 1
        FROM Purchasing.PurchaseOrderLines AS POL
        JOIN Purchasing.PurchaseOrders AS PO
          ON PO.PurchaseOrderID = POL.PurchaseOrderID
        WHERE PO.SupplierID = S.SupplierID
          AND POL.StockItemID IN (SELECT SI2.StockItemID FROM Warehouse.StockItems AS SI2 WHERE SI2.ColorID = SI.ColorID)
    )
);
```

**Execution Result (screenshot):**

![](/mnt/data/6526FE91-EE54-4510-B472-A5945D3F2480.jpeg)

## Proposition 4 — Correlated subqueries: last order per customer

**Observation:** Non-empty: one row per customer showing latest order.

```sql
SELECT O.CustomerID, O.OrderID, O.OrderDate
FROM Sales.Orders AS O
WHERE O.OrderDate =
(
    SELECT TOP (1) O2.OrderDate
    FROM Sales.Orders AS O2
    WHERE O2.CustomerID = O.CustomerID
    ORDER BY O2.OrderDate DESC, O2.OrderID DESC
)
AND O.OrderID =
(
    SELECT TOP (1) O3.OrderID
    FROM Sales.Orders AS O3
    WHERE O3.CustomerID = O.CustomerID
    ORDER BY O3.OrderDate DESC, O3.OrderID DESC
);
```

**Execution Result (screenshot):**

![](/mnt/data/2088A287-86CD-4F41-A9A4-0A2CDA5098E9.jpeg)

## Proposition 5 — CTEs: cities with more customers than suppliers

**Observation:** Non-empty: cities where NumCustomers > NumSuppliers.

```sql
WITH CustCounts AS
(
    SELECT DeliveryCityID AS CityID, COUNT(*) AS NumCustomers
    FROM Sales.Customers
    GROUP BY DeliveryCityID
),
SuppCounts AS
(
    SELECT DeliveryCityID AS CityID, COUNT(*) AS NumSuppliers
    FROM Purchasing.Suppliers
    GROUP BY DeliveryCityID
)
SELECT COALESCE(c.CityID, s.CityID) AS CityID,
       COALESCE(c.NumCustomers, 0) AS NumCustomers,
       COALESCE(s.NumSuppliers, 0) AS NumSuppliers
FROM CustCounts AS c
FULL JOIN SuppCounts AS s
  ON s.CityID = c.CityID
WHERE COALESCE(c.NumCustomers, 0) > COALESCE(s.NumSuppliers, 0);
```

**Execution Result (screenshot):**

![](/mnt/data/F1701899-E399-4154-AA15-4F8B6F808821.jpeg)

## Proposition 6 — CROSS APPLY: top 1 most expensive line per order

**Observation:** Non-empty: exactly one most-expensive line per order.

```sql
SELECT O.OrderID, A.StockItemID, A.UnitPrice, A.Quantity, A.LineProfit
FROM Sales.Orders AS O
CROSS APPLY
(
    SELECT TOP (1)
           OL.StockItemID,
           OL.UnitPrice,
           OL.Quantity,
           (OL.UnitPrice * OL.Quantity) AS LineProfit
    FROM Sales.OrderLines AS OL
    WHERE OL.OrderID = O.OrderID
    ORDER BY (OL.UnitPrice * OL.Quantity) DESC, OL.OrderLineID DESC
) AS A;
```

**Execution Result (screenshot):**

![](/mnt/data/CA228888-52C8-4509-B9C5-97BEA5718E55.jpeg)

## Proposition 7 — OUTER APPLY: first invoice per customer (include customers with none)

**Observation:** Non-empty overall; customers without invoices would show NULLs.

```sql
SELECT C.CustomerID, C.CustomerName, A.InvoiceID, A.InvoiceDate
FROM Sales.Customers AS C
OUTER APPLY
(
    SELECT TOP (1) I.InvoiceID, I.InvoiceDate
    FROM Sales.Invoices AS I
    WHERE I.CustomerID = C.CustomerID
    ORDER BY I.InvoiceDate ASC, I.InvoiceID ASC
) AS A;
```

**Execution Result (screenshot):**

![](/mnt/data/0F63FB03-1A6D-4D55-A555-CBF90B51A4F5.jpeg)

## Proposition 8 — EXISTS + NOT EXISTS: items with warehouse activity but never sold

**Observation:** EMPTY: every item with warehouse activity was sold at least once.

```sql
SELECT SI.StockItemID, SI.StockItemName
FROM Warehouse.StockItems AS SI
WHERE EXISTS
(
    SELECT 1
    FROM Warehouse.StockItemTransactions AS T
    WHERE T.StockItemID = SI.StockItemID
)
AND NOT EXISTS
(
    SELECT 1
    FROM Sales.InvoiceLines AS IL
    WHERE IL.StockItemID = SI.StockItemID
);
```

**Execution Result (screenshot):**

![](/mnt/data/81E1C7A0-2D46-4C11-A12C-967B870A9D51.jpeg)

## Proposition 9 — Nested NOT EXISTS: countries where every customer placed ≥ 1 order

**Observation:** Non-empty: list of countries where all customers have ≥ 1 order.

```sql
SELECT Ctry.CountryName
FROM Application.Countries AS Ctry
WHERE NOT EXISTS
(
    SELECT 1
    FROM Sales.Customers AS C
    JOIN Application.Cities AS Ci
      ON Ci.CityID = C.DeliveryCityID
    JOIN Application.StateProvinces AS SP
      ON SP.StateProvinceID = Ci.StateProvinceID
    WHERE SP.CountryID = Ctry.CountryID
      AND NOT EXISTS (SELECT 1 FROM Sales.Orders AS O WHERE O.CustomerID = C.CustomerID)
);
```

**Execution Result (screenshot):**

![](/mnt/data/76AE1C20-82CB-42C0-AB63-C4EFEBE69B02.jpeg)

## Proposition 10 — Derived table + APPLY: 3 most recent orders per customer with totals

**Observation:** Non-empty: up to three latest orders per customer with totals.

```sql
SELECT C.CustomerID, C.CustomerName, A.OrderID, A.OrderDate, A.OrderTotal
FROM Sales.Customers AS C
CROSS APPLY
(
    SELECT TOP (3) O.OrderID, O.OrderDate,
           (
               SELECT SUM(OL.UnitPrice * OL.Quantity)
               FROM Sales.OrderLines AS OL
               WHERE OL.OrderID = O.OrderID
           ) AS OrderTotal
    FROM Sales.Orders AS O
    WHERE O.CustomerID = C.CustomerID
    ORDER BY O.OrderDate DESC, O.OrderID DESC
) AS A
ORDER BY C.CustomerID, A.OrderDate DESC, A.OrderID DESC;
```

**Execution Result (screenshot):**

![](/mnt/data/109991DC-D07A-4B89-BDEA-34B82E9BFF8C.jpeg)